<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creating-model" data-toc-modified-id="Creating-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creating model</a></span></li><li><span><a href="#Computation" data-toc-modified-id="Computation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Computation</a></span></li></ul></div>

## Creating model

In [ ]:
from pathlib import Path

# Initialization
from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

# containers
#   * bodies
bodies = pre.avatars()
#   * materials
mats   = pre.materials()
#   * materials
mods   = pre.models()
#   * see tables
svs    = pre.see_tables()
#   * contact laws
tacts  = pre.tact_behavs()

# 2D
dim = 2

# mod & mat
mo = pre.model(name='M2D_L',physics='MECAx',element='Q4xxx',dimension=2, external_model='MatL_',
               kinematic='small',material='elas_',anisotropy='iso__',mass_storage='coher')
mods+=mo
ma = pre.material(name='steel',materialType='ELAS',elas='standard',
                  young=0.1e+15,nu=0.2,anisotropy='isotropic',
                  density=0.25e+4)

mats.addMaterial(ma)

#  meshing
m1 = pre.buildMesh2D('Q4', x0=1.5, y0=1.5, lx=1., ly=1., nb_elem_x=10, nb_elem_y=10)

#avatar
b1=pre.buildMeshedAvatar(mesh=m1, model=mo, material=ma)

# lets embed nodes contained in group 'down'
b1.imposeDrivenDof(group='down',component=[1,2],dofty='vlocy')

# lets apply force to nodes contained in group 'up'
# Two equivalent solutions

#b1.imposeDrivenDof(group='up',component=2,dofty='force',ct=10., rampi=0., ramp=10.)

b1.imposeDrivenDof(group='up',component=2,dofty='force',description='evolution',evolutionFile='Fy.txt')
with open(datbox/'Fy.txt','w') as ofile:
    ofile.write('%12.5e %12.5e\n' % (0.,0.))
    ofile.write('%12.5e %12.5e\n' % (0.1,1e10))
    ofile.write('%12.5e %12.5e\n' % (100.,1e10))

bodies+=b1

In [ ]:
# Lets generate commands for post-processing
post = pre.postpro_commands()
#
my_command=pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)

# Lets write files
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post,
                gravy=[0.,0.,0.], datbox_path=datbox)

In [ ]:
pre.visuAvatars(bodies)

## Computation

In [ ]:
from pylmgc90.chipy import computation

# space dimension
dim = 2

# modeling hypothesis: 1 = plain strain
mhyp = 1

# time evolution parameters
dt = 1e-3
nb_steps = 500

# theta integrator parameter
theta = 0.5

# interaction parameters
Rloc_tol = 5.e-2

# nlgs parameters
tol = 1e-4
relax = 1.0
norm = 'Quad '
gs_it1 = 50
gs_it2 = 10
stype='Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

hfile = 'lmgc90.h5'


computation.initialize(dim, dt, theta, mhyp, h5_file=hfile, deformable=True)

for k in range(1, nb_steps+1):
    if k%50==0:
        print( f"computing step {k}")
    computation.one_step(stype, norm, tol, relax, gs_it1, gs_it2,
                         freq_write, freq_display                )

computation.finalize()

In [ ]:
#!paraview